In [ ]:
import requests
import pandas as pd
import os
from api_key import census_api

In [ ]:
industry_worker_type = pd.DataFrame()

In [ ]:
worker_type_list = ['Employee of private company workers', 'Self-employed in own incorporated business workers','Private not-for-profit wage and salary workers', 
                    'Local, state, and federal government workers', 'Self-employed in own not incorporated business workers and unpaid family workers']

In [ ]:
indices_to_skip = [55, 82, 109, 136]

In [5]:
for worker_index in range(len(worker_type_list)):
    if worker_index in indices_to_skip:
        continue

    for skip_index in indices_to_skip:
        if worker_index >= skip_index:
            worker_index += 1
    
    worker_int = str(worker_index + 29).zfill(2)  
    try:
        worker_data = pd.DataFrame()  
        print(f"Fetching data for  {worker_type_list[worker_index]}...")
        for year in range(2014, 2023):
            print(f"Fetching data for year {year}...")
            url = f'https://api.census.gov/data/{year}/acs/acs1?get=NAME,B24070_0{worker_int}E&for=place:*&key={census_api}'
            response = requests.get(url)
            if response.status_code == 200:
                json_response = response.json()
                data = pd.DataFrame(json_response[1:], columns=json_response[0])
                data = data.rename(columns={f'B24070_0{worker_int}E': 'Worker Count'})
                data = data.dropna()
                data = data.loc[data['Worker Count'] != '0']
                data['Class of Worker'] = worker_type_list[worker_index]
                data['Worker Count'] = data['Worker Count'].astype(int)
                data = data.sort_values('Worker Count', ascending=False)
                data['Year'] = str(year)
                worker_data = pd.concat([worker_data, data], ignore_index=True) 
                print(f"Data for year {year} fetched successfully.")
                print(f"Size of worker data: {worker_data.shape}")
                print(f"Size of data after concatenation: {data.shape}")
            else:
                print(f"Error fetching data for year {year}: Status code {response.status_code}")
        industry_worker_type = pd.concat([industry_worker_type, worker_data], ignore_index=True)
        print(f"Size of combined data: {industry_worker_type.shape}")
    except Exception as e:
        print(f'Error fetching data for number of works by worker type {worker_type_list[worker_index]}: {e}')
print(industry_worker_type)

Fetching data for  Employee of private company workers...
Fetching data for year 2014...
Error fetching data for number of works by worker type Employee of private company workers: 'Total Households'
Fetching data for  Self-employed in own incorporated business workers...
Fetching data for year 2014...
Error fetching data for number of works by worker type Self-employed in own incorporated business workers: 'Total Households'
Fetching data for  Private not-for-profit wage and salary workers...
Fetching data for year 2014...
Error fetching data for number of works by worker type Private not-for-profit wage and salary workers: 'Total Households'
Fetching data for  Local, state, and federal government workers...
Fetching data for year 2014...
Error fetching data for number of works by worker type Local, state, and federal government workers: 'Total Households'
Fetching data for  Self-employed in own not incorporated business workers and unpaid family workers...
Fetching data for year 2014

: 

In [ ]:
industry_worker_type